***
# **Classificação de Assuntos Processuais** <img src="https://cdn-icons-png.flaticon.com/512/3292/3292568.png" width="3%">
***

Neste projeto, vamos classificar o assunto processual de acordo com o texto da sua petição inicial. O grande objetivo por trás disso será evitar que o advogado possa errar o assunto ao qual submete esse processo, e que seja gasto tempo dos servidores lendo a petição inicial e corrigindo seu devido assunto para que ele tenha seu trâmite ocorra de forma correta. 

<br>
<p align=center>
<img src="https://cdn-icons-png.flaticon.com/512/2912/2912872.png" width="30%"></p>
<br>

Esse projeto se ganha importância quando vemos a quantidade de processos abertos por ano. A Justiça abriu <a href="https://veja.abril.com.br/coluna/radar/judiciario-teve-277-milhoes-de-casos-abertos-no-ultimo-ano/">27,7 milhões</a> de novos casos em 2021, alta de 10,4% em relação ao observado em 2020. O dado faz parte do relatório “Justiça em Números”. Considerando somente as ações ajuizadas pela primeira vez em 2021, o número vai para 19,1 milhões de novos processos.

Esse projeto não vai acabar de vez com a morosidade da justiça, mas faz parte de um pequeno esforço dentro de tantos outros esforços que prometem contribuir cada vez mais por uma justiça melhor. 


***
## **Carregamento de dependências** <img src="https://cdn-icons-png.flaticon.com/512/4461/4461808.png" width="3%">
***

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev -q

Reading package lists...
Building dependency tree...
Reading state information...
pkg-config is already the newest version (0.29.1-0ubuntu4).
python3-dev is already the newest version (3.8.2-0ubuntu2).
python3-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 46.0 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler-cpp0v5 amd64 0.86.1-0ubuntu1.1 [35.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler-cpp-dev amd64 0.86.1-0ubuntu1.1 [10.4 kB]
Fetched 46.0 kB in 0s (298 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the

In [3]:
!pip install pdftotext -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [54]:
import glob
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pdftotext
import re
import seaborn as sns
from dataclasses import dataclass

from imblearn.under_sampling import RandomUnderSampler
from pandas.core.common import random_state
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from xgboost import XGBClassifier


# filtrar mensagens de warning
import warnings
warnings.filterwarnings('ignore')


In [5]:
nltk.download('all')

stopwords = nltk.corpus.stopwords.words('portuguese')

stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [245]:
class limpeza():
    stopwords = ['br', 'pág', 'listView', 'https', 'http', 'advogado', 'tabela', 'seguir', 'administrativo', 'processo',
                 'numbr','numbrª', 'ª', 'nº', 'juízo', 'Federal', 'ofício', 'geral', 'defensoria', 'pública', 'união',
                 'justiça', 'federal', 'região', 'pje', 'processo', 'judicial', 'vara', 'subseção', 'judiciária',
                 'juizado', 'especial' , 'cível', 'seção', 'excelentíssimo', 'senhor', 'doutor', 'juiz', 'acórdão',
                 'classe', 'procedimento', 'órgão', 'distribuição', 'excelentissimo', 'gerador', 'fato',
                 'brasileira', 'divorciada', 'servidora', 'cpf', 'residente', 'domiciliada', 'rua',
                 'constituído', 'através', 'procuração', 'anexo', 'vem', 'perante', 'vossa', 'excelência', 'todo', 'acato', 'ajuizar',
                 'presente', 'obrigação', 'fazer', 'pessoa', 'jurídica', 'direito', 'público', 'inscrita', 'cnpj', 'representada', 'localizada',
                 'estado', 'rio', 'grande', 'norte',"oab"]
    def __init__ (self,text):
      self.text=text
      #super.__init__(stopwords)
    def print_texto(self):
      print(self.text)
    def clean_telefone(self):
      regex_telefone = r"\b(\d{2}\)?\s?)?\d{4,5}-?\d{4}\b"
      self.text=re.sub(regex_telefone,"telefone",self.text)
    def clean_espaco(self):
      self.text = re.sub(r'\s+', ' ', self.text) # Substitui o espaço em branco entre os termos por um único espaço
      self.text = re.sub(r'^\s+|\s+?$', '', self.text) # Remova os espaços em branco iniciais e finais
    def clean_urls(self):
      regex_url=r'https?://\S+|www\.\S+'
      self.text = re.sub(regex_url, 'webaddress', self.text) # Substitui URLs por 'webdress'

    def clean_email(self):
      regex_email=r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
      self.text = re.sub(regex_email, 'emailaddress', self.text) # Substitui os endereços de e-mail por 'emailaddress'
       
    def clean_dinheiro(self):
      self.text = re.sub(r'£|\$|r\$', 'moneysymb', self.text) # Substitui os símbolos de dinheiro por 'moneysymb' 
    def clean_pontuacao(self):
      self.text = re.sub('_{1,}', ' ', self.text) #Substitua os '_' por ' '
      self.text = re.sub('[^\w\d\s]', ' ', self.text) # Remover pontuação   
    # Remove as stopwords
    def remove_stopwords(self):
      self.text = ' '.join(word for word in self.text.split(' ') if ((word not in self.stopwords) and (len(word) > 2 )))  # retira stopwords e palavras com menos de 3 letras

    # Função de stematizar o texto, deixando as palavras somente com radicais
    def stemming(self):
      palavras=[]
      for w in self.text.split():
          palavras.append(stemmer.stem(w))
      self.texto_stemming= " ".join(palavras)
    
    def aplicar_limpeza(self):
      self.remove_header_footer()
      self.clean_espaco()
      self.clean_urls()
      self.clean_email()
      self.clean_dinheiro()
      self.clean_telefone()
      self.clean_pontuacao()
      self.remove_stopwords()
      self.stemming()
      

    # Função de remover cabeçalho e rodapé
    def remove_header_footer(self):
      '''Remoção dos cabeçalhos e rodapé do texto da petição'''
      #Verifica se a entrada em uma string ou uma lista
      if isinstance(self.text, str) == False:
          return 'Entrada não é uma str'
      
      list_of_pages = [self.text]

      # Dividi o texto em cada quebra de linha '\n'
      input_split = [
          p.split("\n") for p in list_of_pages
      ]
      #Removendo espaços em branco extras
      for i in range(len(input_split)):
          input_split[i] = list(map(str.strip, input_split[i]))


      #Contar o numero de ocorrencia de cada linha
      counts = {}
      for i in input_split[0]:
          counts[i] = counts.get(i, 0) + 1
      commum_lines = [key for key, value in counts.items() if value >= 2]

      input_split_fixed = []

      for page in input_split:
          text = [i for i in page if i not in commum_lines]
          input_split_fixed.append("\n".join(text))

      self.text='\n'.join(input_split_fixed)
        


In [260]:
#Funções usadas no projeto Para Transformar de pdf para Texto e no DataFrame
def transforma_dataframe_novo(lista_arquivos, classe):
  lista_strings = []
  for peticao in lista_arquivos:
    with open(peticao, "rb") as f:
      pdf = pdftotext.PDF(f)
      
    lista_strings.append("\n\n".join(pdf))
  for i in range(0,len(lista_strings)):
    texto_limpo = limpeza(lista_strings[i])
    texto_limpo.aplicar_limpeza()
    lista_strings[i]=texto_limpo.text
  classificacao = []
  for i in range(len(lista_arquivos)):
    classificacao.append(classe)

  df = cria_dataframe(lista_arquivos, lista_strings, classificacao)
  return df


def cria_dataframe(lista_arquivos, lista_strings, classificacao):
  dicionario_peticao = {
    'nome_documento': lista_arquivos,
    'conteudo_peticao': lista_strings,
    'classificacao': classificacao,
  }
  
  df_peticao_inicial = pd.DataFrame(dicionario_peticao)
  return df_peticao_inicial

In [234]:
peticao="/content/drive/MyDrive/Peticoes_teste/trf5.peticoes/15 - APOSENTADORIA POR IDADE URBANA - ANALISAR/*pdf"
lista_strings=[]
with open(peticao, "rb") as f:
      pdf = pdftotext.PDF(f)
lista_strings.append("\n\n".join(pdf))

print(lista_strings)

['EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(A) FEDERAL DA 27ª\nVARA FEDERAL DA SUBSEÇÃO JUDICIÁRIA DE PERNAMBUCO.\n              MARIA DE LIMA SA, brasileira, casada, portadora da carteira de\nidentidade registrada sob o n° 10.728.368 SDS/PE, inscrita no CPF sob o nº\n269.689.674-72, residente e domiciliada na Rua Baldomiro Pedro da Silva, n°\n149, Santo Antônio, Ouricuri – PE, CEP:56.200-000 vem a presença de vossa\nexcelência, por meio de suas Advogadas, conforme procuração em anexo,\npropor:\n              AÇÃO PREVIDENCIÁRIA DE APOSENTADORIA POR IDADE\nPELA REGRA DE TRANSIÇÃO DO ART. 18, DA EC 103/2019.\n              Em face do INSTITUTO NACIONAL DO SEGURO SOCIAL (INSS),\nAutarquia Federal, com sede na Rua Engenheiro Baltazar Cavalcanti de Farias,\nCentro, CEP:56200-000, Ouricuri – PE, pelos fatos e fundamentos jurídicos a\nseguir expostos:\n1 – DOS FATOS\n              A Requerente, nascida em 9 de setembro de 1960, contando\natualmente com 62 anos de idade, filiou-se à Previdên

In [235]:
type(lista_strings[0])

str

'EXCELENTÍSSIMO SENHOR DOUTOR JUIZ FEDERAL 27ª VARA FEDERAL SUBSEÇÃO JUDICIÁRIA PERNAMBUCO MARIA LIMA casada portadora carteira identidade registrada sob 728 368 SDS CPF sob 269 689 674 Rua Baldomiro Pedro Silva 149 Santo Antônio Ouricuri CEP 200 000 presença por meio suas Advogadas conforme propor AÇÃO PREVIDENCIÁRIA APOSENTADORIA POR IDADE PELA REGRA TRANSIÇÃO ART 103 2019 face INSTITUTO NACIONAL SEGURO SOCIAL INSS Autarquia com sede Rua Engenheiro Baltazar Cavalcanti Farias Centro CEP 56200 000 Ouricuri pelos fatos fundamentos jurídicos expostos DOS FATOS Requerente nascida setembro 1960 contando atualmente com anos idade filiou Previdência 1993 sendo que até data possui diversos anos contribuição demonstra forma objetiva estes períodos ATIVIDADE INÍCIO FIM TIPO TEMPO ATIVIDADE CONTRIBUIÇÃO ULTRALIMPO 1993 Normal ano EMPREENDIMENTOS 2003 mes dia SERVICOS LTDA ENGENHARIA 2010 Normal ano SERVICOS 2010 mes dia RECOLHIMENTO 2010 Normal ano 2010 mes dia RECOLHIMENTO 2012 Normal ano 2012 

(99)99999-9999.


####Codigo Antigo


In [257]:
#Funções usadas no projeto Para Transformar de pdf para Texto e no DataFrame
def transforma_dataframe_antigo(lista_arquivos, classe):
  lista_strings = []
  for peticao in lista_arquivos:
    with open(peticao, "rb") as f:
      pdf = pdftotext.PDF(f)
      
    lista_strings.append("\n\n".join(pdf))

  classificacao = []
  for i in range(len(lista_arquivos)):
    classificacao.append(classe)

  df = cria_dataframe(lista_arquivos, lista_strings, classificacao)
  return df


def cria_dataframe(lista_arquivos, lista_strings, classificacao):
  dicionario_peticao = {
    'nome_documento': lista_arquivos,
    'conteudo_peticao': lista_strings,
    'classificacao': classificacao,
  }
  
  df_peticao_inicial = pd.DataFrame(dicionario_peticao)
  return df_peticao_inicial




In [258]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/Peticoes_teste/trf5.peticoes/20 - AUXÍLIO-EMERGENCIAL/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_auxilio_emergencial = transforma_dataframe_antigo(lista_arquivos, "auxilio_emergencial")

In [261]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/Peticoes_teste/trf5.peticoes/20 - AUXÍLIO-EMERGENCIAL/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_auxilio_emergencial = transforma_dataframe_novo(lista_arquivos, "auxilio_emergencial")

In [255]:
df_auxilio_emergencial

,nome_documento,conteudo_peticao,classificacao
0,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,Justiça Federal da 5ª Região\n ...,auxilio_emergencial
1,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(A) ...,auxilio_emergencial
2,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,Justiça Federal da 5ª Região\n ...,auxilio_emergencial
3,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO (A) SENHOR (A) JUIZ (A) FEDERAL...,auxilio_emergencial
4,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,D EF EN SO R I A P ÚBLI CA ...,auxilio_emergencial
5,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,\n\n\n\n,auxilio_emergencial
6,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO SENHOR JUIZ FEDERAL DO JUIZADO ...,auxilio_emergencial
7,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO (A) SENHOR (A) DOUTOR (A) JUIZ ...,auxilio_emergencial
8,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,DEFENSORIA PÚBLICA...,auxilio_emergencial
9,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,DEFENSORIA PÚBLICA DA ...,auxilio_emergencial


***
## **Importando os documentos de petição** <img src="https://as2.ftcdn.net/v2/jpg/01/69/33/39/1000_F_169333910_pGApA6p8bYBGq0oNmJYJLOswGfCRZuxE.jpg" width="3%">
***

Foi realizado um trabalho manual dos servidores da justiça em que a cada petição inicial corrigida, a mesma era colocada em uma determinada pasta com o nome do assunto a qual ela pertencia. Devido a isso, foi feito uma engenharia em que foi criado o dataFrame a partir dessa separação de petições por pastas. 

No final, será gerado um arquivo csv para não rodar o script para gerar os dataFrames, e será trabalhado com esse arquivo CSV. Em caso de novas petições chegarem, o processo de gerar o nosso dataFrame poderar ser usado novamente, gerando um novo arquivo csv, ao qual substituiremos. 

In [44]:
def transforma_dataframe(lista_arquivos, classe):
  lista_strings = []
  for peticao in lista_arquivos:
    with open(peticao, "rb") as f:
      pdf = pdftotext.PDF(f)
    lista_strings.append("\n\n".join(pdf))

  classificacao = []
  for i in range(len(lista_arquivos)):
    classificacao.append(classe)

  df = cria_dataframe(lista_arquivos, lista_strings, classificacao)
  return df

def cria_dataframe(lista_arquivos, lista_strings, classificacao):
  dicionario_peticao = {
    'nome_documento': lista_arquivos,
    'conteudo_peticao': lista_strings,
    'classificacao': classificacao,
  }
  
  df_peticao_inicial = pd.DataFrame(dicionario_peticao)
  return df_peticao_inicial

  

In [45]:
file_path="/content/drive/MyDrive/Peticoes_teste/trf5.peticoes"

In [51]:
df_idade_hibrida

,nome_documento,conteudo_peticao,classificacao
0,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(A) ...,idade_hibrida
1,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,AO JUÍZO DA VARA FEDERAL DA SUBSEÇÃO JUDICIÁRI...,idade_hibrida
2,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,AO DOUTO JUIZO DA 6ª VARA FEDERAL SEÇÃO JUDI...,idade_hibrida
3,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,DOUTO JUÍZO DA VARA FEDERAL – JUIZADO ES...,idade_hibrida
4,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO SR. DR. JUIZ FEDERAL DO JUIZADO...,idade_hibrida
5,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,Justiça Federal da 5ª Região\n ...,idade_hibrida
6,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,Justiça Federal da 5ª Região\n ...,idade_hibrida
7,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO SENHOR DOUTOR JUIZ FEDERAL DA 6...,idade_hibrida
8,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXMO (A). SR (A). DR (A). JUIZ (A) DA ___...,idade_hibrida
9,/content/drive/MyDrive/Peticoes_teste/trf5.pet...,EXCELENTÍSSIMO (A) SENHOR (A) DOUTOR (A) JUIZ...,idade_hibrida


In [29]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/20 - AUXÍLIO-EMERGENCIAL/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_auxilio_emergencial = transforma_dataframe(lista_arquivos, "auxilio_emergencial")

In [30]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/20 - SAÚDE - FORNECIMENTO DE MEDICAMENTOS/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_fornecimento_medicamentos = transforma_dataframe(lista_arquivos, "fornecimento_medicamentos")

In [31]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/21 - AUXÍLIO-DOENÇA/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_auxilio_doenca = transforma_dataframe(lista_arquivos, "auxilio_doenca")

,nome_documento,conteudo_peticao,classificacao


In [12]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/23 - APOSENTADORIA POR IDADE URBANA/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_aposentadoria_idade_urbana = transforma_dataframe(lista_arquivos, "aposentadoria_idade_urbana")

In [13]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/23 - TRIB - CONTR. PREV. PAGAS ALÉM DO TETO/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_prev_pagas_alem_do_teto = transforma_dataframe(lista_arquivos, "prev_pagas_alem_do_teto")

In [14]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/25 - APOSENTADORIA POR TEMPO DE CONTRIBUICAO/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_aposentadoria_tempo_contribuicao = transforma_dataframe(lista_arquivos, "aposentadoria_tempo_contribuicao")

In [15]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/25 - AUXILIO RECLUSAO/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_auxilio_reclusao = transforma_dataframe(lista_arquivos, "auxilio_reclusao")

In [16]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/25 - SALARIO MATERNIDADE - URBANA/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_salario_maternidade_urbana = transforma_dataframe(lista_arquivos, "salario_maternidade_urbana")

In [17]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/26 - ACAO COBRANCA DPVAT/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_acao_cobranca_dpvat = transforma_dataframe(lista_arquivos, "acao_cobranca_dpvat")

In [18]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/29 - REVISIONAL FGTS/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_revisional_fgts = transforma_dataframe(lista_arquivos, "revisional_fgts")

In [19]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/33 - AUXÍLIO-DOENÇA E APOSENTADORIA POR INVALIDEZ/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_aux_doenca_aposent_invalidez = transforma_dataframe(lista_arquivos, "aux_doenca_aposent_invalidez")

In [20]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/35 - PENSAO POR MORTE/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_pensao_p_morte = transforma_dataframe(lista_arquivos, "pensao_p_morte")

In [21]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/45 - APOSENTADORIA IDADE RURAL/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_aposen_idade_rural = transforma_dataframe(lista_arquivos, "aposen_idade_rural")

In [22]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/57 - SALARIO MATERNIDADE RURAL/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_salario_maternidade_rural = transforma_dataframe(lista_arquivos, "salario_maternidade_rural")

In [23]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/48 - APOSENTADORIA ESPECIAL/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_aposent_especial = transforma_dataframe(lista_arquivos, "aposent_especial")

In [24]:
lista_arquivos = []
for arquivo in glob.glob(r'/content/drive/MyDrive/trf5.peticoes/65 - BENEFICIO ASSISTENCIAL - LOAS/*pdf'):
  if arquivo not in lista_arquivos:
    lista_arquivos.append(arquivo)

df_benef_assist_loas = transforma_dataframe(lista_arquivos, "benef_assist_loas")

In [25]:
# Concatenando os dataframes
df_peticoes_iniciais = pd.concat(
                                  [
                                  df_idade_hibrida,
                                  df_auxilio_emergencial, 
                                  df_fornecimento_medicamentos,
                                  df_auxilio_doenca,
                                  df_aposentadoria_idade_urbana, 
                                  df_prev_pagas_alem_do_teto, 
                                  df_aposentadoria_tempo_contribuicao,
                                  df_auxilio_reclusao, 
                                  df_salario_maternidade_urbana,
                                  df_acao_cobranca_dpvat, 
                                  df_revisional_fgts, 
                                  df_aux_doenca_aposent_invalidez, 
                                  df_pensao_p_morte, 
                                  df_aposen_idade_rural,
                                  df_aposent_especial,
                                  df_salario_maternidade_rural,
                                  df_benef_assist_loas
                                  ], 
                                 ignore_index=True
                                 )

In [26]:
df_peticoes_iniciais.to_csv(r'/content/drive/MyDrive/trf5.peticoes/DataFrame_10_02_2023', index=False)

OSError: ignored

In [ ]:
df_peticoes_iniciais['conteudo_peticao'].filter(items=[78], axis=0)

In [ ]:
df_peticoes_iniciais

***
## **Pré-processamento** <img src="https://cdn-icons-png.flaticon.com/512/1556/1556231.png" width="3%"> 
***

Nesta etapa vamos preparar os dados e usar Processamento de Linguagem Natural(PLN ou NLP) para melhorar nosso modelo. 

In [ ]:
# Aqui carregamos o CSV sempre atualizado
df_peticoes_iniciais = pd.read_csv('/content/drive/MyDrive/Peticoes_teste/trf5.peticoes/DataFrame_10_02_2023')

Finalmente, nosso dataFrame Inicial foi criado

In [ ]:
# Imprime 3 linhas dos nossos dados
df_peticoes_iniciais.head(3)

In [ ]:
df_peticoes_iniciais.dtypes

In [ ]:
# Verificando a distribuição de classe

print(df_peticoes_iniciais['classificacao'].value_counts())

Nossa variável alvo, a classificação, é uma variável categórica. Se a submetermos dessa forma no nosso algoritmo de machine learning, receberemos um erro. Para ser entendido por nosso modelo, precisamos atribuir um valor para cada classe. 

In [ ]:
# Lista de assuntos processuais sem repetição
classificacao_peticoes = df_peticoes_iniciais['classificacao'].unique()

# Criar dicionário python e atribuir um número unico a cada assunto processual
classificacao_peticoes_dicionario = {}
var = 0
for tipo_peticao in classificacao_peticoes:
  classificacao_peticoes_dicionario[tipo_peticao] = var
  var +=1

# criação de uma label com a classe assunto defida por números
df_peticoes_iniciais['label_num'] = df_peticoes_iniciais['classificacao'].map(
    classificacao_peticoes_dicionario
)

label_ordinal_encoding = df_peticoes_iniciais[['classificacao','label_num']]
label_ordinal_encoding


In [ ]:
classificacao_peticoes_dicionario

Aqui, nós fazemos um tratamento para limpar um pouco o texto de possíveis elementos que não seja do nosso interesse. Nós removemos um conjunto de palavras conhecido como stopwords, retiramos os vários espaços em branco dos textos, caracteres especiais, e fazemos algumas substituições. 

Outra coisa, também que fazemos, é retirar o cabeçalho e o rodapé, que pode causar algum tipo de viés caso tivermos muitas petições feitas de um escritório de advocacia em particular. 

Por último e não menos importante, fazemos a stematização dos termos, reduzindo a sua forma em radical, em que palavras como “juiz”, “juízes” e “juízas” serão contabilizadas como apenas uma, seu radical, “juiz”.


In [ ]:


# Função para limpar algumas sujeitas do texto
def clean_text(text):
  text = str(text).lower()
  text = re.sub('https?://\S+|www\.\S+', 'webaddress', text) # Substitui URLs por 'webdress'
  text = re.sub('^.+@[^\.].*\.[a-z]{2,}$', 'emailaddress', text) # Substitui os endereços de e-mail por 'emailaddress'
  text = re.sub('£|\$|r\$', 'moneysymb', text) # Substitui os símbolos de dinheiro por 'moneysymb' 
  text = re.sub('^\(?[\d]{2}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumbr', text) # Substitui números de telefone de 10 dígitos (os formatos incluem parênteses, espaços, sem espaços, traços) por 'phonenumber'
  text = re.sub('\d+(\.\d+)?', '', text) # Substitua os números por 'numbr'
  text = re.sub('_{1,}', ' ', text) #Substitua os '_' por ' '
  text = re.sub('[^\w\d\s]', ' ', text) # Remover pontuação
  text = re.sub('\s+', ' ', text) # Substitui o espaço em branco entre os termos por um único espaço
  text = re.sub('^\s+|\s+?$', '', text) # Remova os espaços em branco iniciais e finais
  return text


# Remove as stopwords
def remove_stopwords(text):
  text = ' '.join(word for word in text.split(' ') if ((word not in stopwords) and (len(word) > 2 )))  # retira stopwords e palavras com menos de 3 letras
  return text


# Função de stematizar o texto, deixando as palavras somente com radicais
def stemming(text):
    palavras=[]
    for w in text.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))


# Função de remover cabeçalho e rodapé
def remove_header_footer(input_data):
    '''
        Remoção dos cabeçalhos e rodapé do texto da petição
    '''
    #Verifica se a entrada em uma string ou uma lista
    if isinstance(input_data, str) == False:
        return 'Entrada não é uma str'
    
    list_of_pages = [input_data]

    # Dividi o texto em cada quebra de linha '\n'
    input_split = [
        p.split("\n") for p in list_of_pages
    ]
    #Removendo espaços em branco extras
    for i in range(len(input_split)):
        input_split[i] = list(map(str.strip, input_split[i]))


    #Contar o numero de ocorrencia de cada linha
    counts = {}
    for i in input_split[0]:
        counts[i] = counts.get(i, 0) + 1
    commum_lines = [key for key, value in counts.items() if value >= 2]

    input_split_fixed = []

    for page in input_split:
        text = [i for i in page if i not in commum_lines]
        input_split_fixed.append("\n".join(text))

    return '\n'.join(input_split_fixed)

In [ ]:
# Aplicação de cada função em cada texto do dataFrame
df_peticoes_iniciais['text_clean'] = df_peticoes_iniciais['conteudo_peticao'].apply(remove_header_footer)

df_peticoes_iniciais['text_clean'] = df_peticoes_iniciais['text_clean'].apply(clean_text)

df_peticoes_iniciais['text_clean'] = df_peticoes_iniciais['text_clean'].apply(remove_stopwords)

df_peticoes_iniciais['text_clean'] = df_peticoes_iniciais['text_clean'].apply(stemming)

In [ ]:
# Imprime a coluna do conteudo da petição e a coluna do texto limpo já tratado e pronto para treino
df_peticoes_iniciais[['conteudo_peticao', 'text_clean']] 

## **Train test split** <img src="https://cdn-icons-png.flaticon.com/512/1610/1610654.png" width="3%">

Agora, vamos separar os dados de treino e os dados de teste. Para isso, usei a função train_test_split da biblioteca scikit-learn, uma das mais famosas no ambito de Machine Learning. Separando a base de treino e de teste, configurei o test size em 20%, ou seja, 80% do dataset será utilizado para o algorítimo aprender e 20% para testar o quão esse algorítimo aprendeu de fato.

In [ ]:
X= df_peticoes_iniciais.text_clean
y= df_peticoes_iniciais.label_num

In [ ]:
# Separa os dados de treino e de teste


X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20, # 20% de dados de teste
    random_state=42,
    stratify=df_peticoes_iniciais.label_num
)

In [ ]:
# Imprime o shape 
print("X_train:", X_train.shape)

print("X_test:", X_test.shape)

print("y_train:", y_train.shape)

print("y_test:", y_test.shape)

print("df_peticoes_iniciais:", X.shape)

print("df_peticoes_iniciais.label_num:",y.shape)


In [ ]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

print("Shape of y_train: ", X_train.shape)
print("Shape of y_test: ", X_test.shape)

In [ ]:
# imprime a quantidade de dados para treino
y_train.value_counts()/ y_train.shape[0]
y_train.value_counts()

In [ ]:
print("df_peticoes_iniciais:\t\t",df_peticoes_iniciais.shape)

print("X_train:\t\t",X_train.shape)

print("y_train:\t\t",y_train.shape)

print("X_train (reshape):\t",X_train.values.reshape(-1,1).shape)



Porém, como o dataset está desbalanceado, há uma tendência que o algorítimo aprenda mais sobre a classe majoritária que a classe minoritária. Isso, consequentemente, fará com que a classe majoritária tenha uma excelente acurácia, e a classe minoritária tenha uma acurácia bem ruim. 

Para resolver isso, vamos usar a técnica de undersampling, que consiste em pegar amostras aleatórias da classe majoritária e igualar em número a classe minoritária. Para isso, usamos a classe RandomUnderSampler do pacote imbalanced-learn.

In [ ]:
'''
o scikit-learn espera duas coisas nesta etapa:

-Que a variável X seja sempre um np.array com duas dimensões
-Que a variável y seja sempre um np.array com uma dimensão

 Foi preciso mudar a dimensão do array para atender a condição acima, por isso foi usado o .reshape(-1,1).
'''
X_train = X_train.values.reshape(-1,1)

In [ ]:
# Definindo o UnderSamplere e aplicando para o x e o y de treino
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)

print(y_res.value_counts())



Vale a pena ressaltar, esses dados que usamos o undersampling, será utilizado apenas para treinar nosso modelo, mas na hora de validar, usaremos os dados de validação normal. Se utilizarmos esses mesmos dados agora balanceados para fazer a validação, estaremos distorcendo a realidade, pois alguns assuntos tem uma ocorrência maior do que outros. 

In [ ]:
# print(X_res.value_counts())

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

***
## **Treinamento e Predição** <img src="https://cdn-icons-png.flaticon.com/512/3273/3273713.png" width="3%">
***

### **Tentativa 1**: KNN

Usando o módulo pipeline sklearn cria um pipeline de classificação para classificar as Petições. 

Foi usado TF-IDF para pré-processar o texto.

Foi usado Unigram, Bigrams e trigrams.

Foi usado **KNN** como o classificador

In [ ]:
#1. Criar um objeto de pipeline
pipeline_vizinhos_proximos = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer(max_features=650, ngram_range = (1, 3))),    
     ('KNN', KNeighborsClassifier(
         n_neighbors=18, #weights='distance',
         algorithm='kd_tree'
     ))         
])

#2. Ajuste(fit) com X_train e y_train
pipeline_vizinhos_proximos.fit(X_res.flatten(), y_res)


#3. Obter previsões para X_test e armazenar em y_pred
y_pred_vizinhosproximos = pipeline_vizinhos_proximos.predict(X_test)


# imprimir relatório 
print(classification_report(y_test, y_pred_vizinhosproximos))


# Imprime a matriz de confusão 
mat = confusion_matrix(y_test, y_pred_vizinhosproximos)

sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False,
            xticklabels=list(classificacao_peticoes_dicionario.keys()),yticklabels=list(classificacao_peticoes_dicionario.keys()))
plt.xlabel('True Label')
plt.ylabel("Predicted Label")
plt.show()

In [ ]:
# Lista de petições e numero de tipos de petições existentes
listOfKeys = classificacao_peticoes_dicionario.keys()
listOfKeys=list(listOfKeys)
print(listOfKeys)
print(len(listOfKeys))

In [ ]:
classificacao_peticoes_dicionario

### **Tentativa 2**: Random Forest

Usando o módulo pipeline sklearn foi criado um pipeline para classificar as petições.

Foi usado TF-IDF para pré-processar o texto.

Foi usado Unigram e Bigrams.

Foi usado **Random Forest** como classificador.


In [ ]:
#1. Criar um objeto de pipeline
pipeline_random_forest = Pipeline([
 
    ('vectorizer_tfidf',TfidfVectorizer(max_features=650, ngram_range = (2, 3))),     
     ('Random Forest', RandomForestClassifier(n_estimators=600, min_samples_leaf=2, 
                                              random_state=0, n_jobs=-1))         
])

#2. Ajuste(fit) com X_train e y_train
pipeline_random_forest.fit(X_res.flatten(), y_res)


#3. Obter previsões para X_test e armazenar em y_pred
y_pred_rf = pipeline_random_forest.predict(X_test)

# imprimir relatório 
print(classification_report(y_test, y_pred_rf))

# Imprime a matriz de confusão 
mat = confusion_matrix(y_test, y_pred_rf)

sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False,
            xticklabels=list(classificacao_peticoes_dicionario.keys()),yticklabels=list(classificacao_peticoes_dicionario.keys()))
plt.xlabel('True Label')
plt.ylabel("Predicted Label")
plt.show()

### **Tentativa 3**: SVM

Usando o módulo pipeline sklearn foi criado um pipeline para classificar as petições.

Foi usado TF-IDF para pré-processar o texto.

Foi usado Bigrams e Trigrams.

Foi usado **SVM** como classificador.


In [ ]:
#1. Criar um objeto de pipeline
pipeline_svc = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer(max_features=650, min_df=13, ngram_range = (2, 3))),        
     ('SVC', SVC(kernel = 'linear', random_state=0))         
])

#2. Ajuste(fit) com X_train e y_train
pipeline_svc.fit(X_res.flatten(), y_res)


#3. Obter previsões para X_test e armazenar em y_pred
y_pred_svm = pipeline_svc.predict(X_test)

# imprimir relatório 
print(classification_report(y_test, y_pred_svm))

# Imprime a matriz de confusão  
mat = confusion_matrix(y_test, y_pred_svm)
sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False,
            xticklabels=list(classificacao_peticoes_dicionario.keys()),yticklabels=list(classificacao_peticoes_dicionario.keys()))
plt.xlabel('True Label')
plt.ylabel("Predicted Label")
plt.show()

In [ ]:
# print(pipeline_svc['vectorizer_tfidf'].vocabulary_)

### **Tentativa 4**: XGBoost

Usando o módulo pipeline sklearn foi criado um pipeline para classificar as petições. 

Foi usado TF-IDF para pré-processar o texto.

Foi usado Unigrams e Bigrams.

use XGBoost como classificador.



In [ ]:



#1. Criar um objeto de pipeline
pipeline_xgboost = Pipeline([
  ('vectorizer_tfidf',TfidfVectorizer(max_features=800, min_df=10, ngram_range = (1, 2))),        
     ('xgboost', XGBClassifier(
            learning_rate=0.03,
            n_estimators=500,
            subsample=0.8, 
            colsample_bynode=0.8,
            num_parallel_tree=4
     ))                
])

#2. Ajuste(fit) com X_train e y_train
pipeline_xgboost.fit(X_res.flatten(), y_res)


#3. Obter previsões para X_test e armazenar em y_pred
y_pred_xgboost = pipeline_xgboost.predict(X_test)
     

# Imprime relatório
print(classification_report(y_test, y_pred_xgboost))

# Imprime a matriz de confusão
mat = confusion_matrix(y_test, y_pred_xgboost)

sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False,
            xticklabels=list(classificacao_peticoes_dicionario.keys()),yticklabels=list(classificacao_peticoes_dicionario.keys()))
plt.xlabel('True Label')
plt.ylabel("Predicted Label")
plt.show()

### **Tentativa 5**: ExtraTreesClassifier

Usando o módulo pipeline sklearn foi criado um pipeline para classificar as petições. 

Foi usado TF-IDF para pré-processar o texto.

Foi usado Unigrams e Bigrams.

Foi usado **ExtraTreesClassifier** como classificador.



In [ ]:
#1. Criar um objeto de pipeline
pipeline_ExtraTreesClassifier = Pipeline([
  ('vectorizer_tfidf',TfidfVectorizer(max_features=600, min_df=10, ngram_range = (1, 2))),        
     ('ExtraTreesClassifier', ExtraTreesClassifier(
            random_state=0,
            n_estimators=300, 
            max_depth=4,
            min_samples_leaf=1,
     ))                
])

#2. Ajuste(fit) com X_train e y_train
pipeline_ExtraTreesClassifier.fit(X_res.flatten(), y_res)


#3. Obter previsões para X_test e armazenar em y_pred
y_pred_ExtraTreesClassifier = pipeline_ExtraTreesClassifier.predict(X_test)

# Imprime relatório
print(classification_report(y_test, y_pred_ExtraTreesClassifier))

# Imprime a matriz de confusão
mat = confusion_matrix(y_test, y_pred_ExtraTreesClassifier)

sns.heatmap(mat.T,square=True,annot=True,fmt='d',cbar=False,
            xticklabels=list(classificacao_peticoes_dicionario.keys()),yticklabels=list(classificacao_peticoes_dicionario.keys()))
plt.xlabel('True Label')
plt.ylabel("Predicted Label")
plt.show()

## **Utilizando Cross-Validation** <img src="https://cdn-icons-png.flaticon.com/512/8003/8003181.png" width="3%">

Como temos uma base de dados desbalanceadas, vamos utilizar StratifiedKFold ao invés do kfold. 

In [ ]:
# Inicializa uma instância de StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
kfold

In [ ]:
# Acurácia média para Vizinhos mais próximos

scores = cross_val_score(pipeline_vizinhos_proximos, X, y, cv=kfold, scoring='accuracy')
print(scores)
print(f'Acurácia média: {scores.mean()}')
print(f'Desvio padrão: {scores.std()}')

In [ ]:
# Acurácia média para random forest

scores = cross_val_score(pipeline_random_forest, X, y, cv=kfold, scoring='accuracy')
print(scores)
print(f'Acurácia média: {scores.mean()}')
print(f'Desvio padrão: {scores.std()}')

In [ ]:
# Acurácia média para SVC

scores = cross_val_score(pipeline_svc, X, y, cv=kfold, scoring='accuracy')
print(scores)
print(f'Acurácia média: {scores.mean()}')
print(f'Desvio padrão: {scores.std()}')

In [ ]:
# Acurácia média para XGBoost

scores = cross_val_score(pipeline_xgboost, X, y, cv=kfold, scoring='accuracy')
print(scores)
print(f'Acurácia média: {scores.mean()}')
print(f'Desvio padrão: {scores.std()}')

In [ ]:
# Acurácia média para ExtraTreesClassifier

scores = cross_val_score(pipeline_ExtraTreesClassifier, X, y, cv=kfold, scoring='accuracy')
print(scores)
print(f'Acurácia média: {scores.mean()}')
print(f'Desvio padrão: {scores.std()}')

In [ ]:
#@title 
# df_peticoes_iniciais.loc[17]['nome_documento']

## **Utilizando roc_auc_score** <img src="https://cdn-icons-png.flaticon.com/512/2738/2738624.png" width="3%">

A *roc_auc_score* é uma métrica de desempenho para problemas de classificação binária que combina a curva ROC (Receiver Operating Characteristic) e o AUC (Area Under the Curve) em uma única medida.

A curva ROC é uma representação gráfica da relação entre a taxa de verdadeiros positivos (TPR) e a taxa de falsos positivos (FPR) para diferentes limiares de classificação. O AUC é a área sob a curva ROC.

A métrica *roc_auc_score* calcula a área sob a curva ROC, ou seja, é a média do AUC em todas as possíveis divisões de limiares de classificação. Um valor de 1 indica uma classificação perfeita e 0.5 indica uma classificação aleatória.

A métrica *roc_auc_score* é uma métrica muito útil para avaliar o desempenho de um modelo de classificação binário, pois leva em conta tanto a sensibilidade (TPR) quanto a especificidade (1-FPR) do modelo. Ele é especialmente útil quando os dados estão desbalanceados, pois não é afetado pelo desequilíbrio de classe.

In [ ]:
listOfValues = classificacao_peticoes_dicionario.values()
listOfValues=list(listOfValues)


In [ ]:
# AUC KNN
y_true_bin = label_binarize(y_test, classes=listOfValues)
y_pred_bin = label_binarize(y_pred_vizinhosproximos, classes=listOfValues)

auc_knn = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
auc_knn

In [ ]:
# AUC Random Forest
y_true_bin = label_binarize(y_test, classes=listOfValues)
y_pred_bin = label_binarize(y_pred_rf, classes=listOfValues)

auc_random_forest = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
auc_random_forest

In [ ]:
# AUC SVM
y_true_bin = label_binarize(y_test, classes=listOfValues)
y_pred_bin = label_binarize(y_pred_svm, classes=listOfValues)

auc_svm= roc_auc_score(y_true_bin, y_pred_bin, average='macro')
auc_svm

In [ ]:
# AUC xgboost
y_true_bin = label_binarize(y_test, classes=listOfValues)
y_pred_bin = label_binarize(y_pred_xgboost, classes=listOfValues)

auc_xgboost = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
auc_xgboost

In [ ]:
# AUC ExtraTreesClassifier
y_true_bin = label_binarize(y_test, classes=listOfValues)
y_pred_bin = label_binarize(y_pred_ExtraTreesClassifier, classes=listOfValues)

auc_ExtraTreesClassifier = roc_auc_score(y_true_bin, y_pred_bin, average='macro')
auc_ExtraTreesClassifier

## **Fazendo Dump** <img src="https://cdn-icons-png.flaticon.com/512/2824/2824710.png" width="3%">

In [ ]:
#@title
# # Fazer upload de petição
# from google.colab import files
# import io, pdftotext
# upload = files.upload()
# filename = list(upload.keys())[0]
# print(filename)

# with open(filename, "rb") as f:
#     pdf = pdftotext.PDF(f)

# texto_documento = "\n\n".join(pdf)
# texto_documento = texto_documento.lower()
# texto_documento = clean_text(texto_documento)
# texto_documento = remove_stopwords(texto_documento)
# #print(texto_documento)

# prediction_teste=predict_category(texto_documento)
# #print(prediction_teste)
# print('Esta petição é sugerida como...')
# if prediction_teste == [0]:
#   print('Aposentadoria por Tempo de Contribuição')
# elif prediction_teste == [1]:
#   print('Aposentadoria Urbana')
# elif prediction_teste == [2]:
#   print('Salario Maternidade Rural')
# elif prediction_teste == [3]:
#   print('Revisional Fgts')
# elif prediction_teste == [4]:
#   print('Pensao po Morte')
# elif prediction_teste == [5]:
#   print('Aposentadoria Idade Rural')
# elif prediction_teste == [6]:
#   print('Aposentadoria Especial')    
# else:
#   print('Beneficio Assistencial Loas')   

     

# def predict_category(s ,model=pipeline_svc):
#     s=remove_header_footer(s)
#     s=clean_text(s)
#     s=remove_stopwords(s)
#     s=stemming(s)
#     pred = model.predict([s])
#     return pred

In [ ]:
# Persistindo o modelo de ML para o disco
import joblib

joblib.dump(pipeline_vizinhos_proximos, '/content/drive/MyDrive/trf5.peticoes/knn_13_02_2023.pkl')

Essa ultima célula é exportando nosso modelo escolhido em um arquivo com extensão .pkl, já pronto para aplicação em produção. Foi decidido exportar o modelo de KNN devido a seus bons desempenhos nas diferentes métricas visualizadas e boa estabilidade na validação cruzada. O trabalho de melhorar cada vez mais continua ... 

<p align=center>
<img src="https://cdn-icons-png.flaticon.com/512/9375/9375341.png" width="30%"></p>
